## Trail for WRF-CHEM, WRF CHEM simulation with dust only for Coimbatore using TWO WAY NESTING TWO files method

- It is based on [this](http://ruc.noaa.gov/wrf/WG11/wrf_tutorial_exercises_v35/exercise_1.html) to run compiled wrf chem in serial mode with only dust.

### WPS:Geogrid

- As per [this](http://ruc.noaa.gov/wrf/WG11/wrf_tutorial_exercises_v35/exercise_1.html) point 7, file ```GEOGRID.TBL.ARW_CHEM``` in WPS geogrid directory was linked as GEOGRID.TBL using GUI of Ubuntu 12.04.
- The namelist.wps used was as follows,

```fortran
&share
max_dom = 4,
start_date 				='2014-06-05_00:00:00','2014-06-05_00:00:00','2014-06-05_00:00:00','2014-06-05_00:00:00'
end_date ='2014-06-05_06:00:00','2014-06-05_06:00:00','2014-06-05_06:00:00','2014-06-05_06:00:00'
interval_seconds = 10800,
io_form_geogrid = 2,
/
&geogrid
parent_id                  = 1, 1, 2, 3
parent_grid_ratio          = 1, 3, 3, 3
i_parent_start             = 1, 26, 22, 22
j_parent_start             = 1, 7, 15, 32
e_we                       = 90, 76, 97, 136
e_sn                       = 85, 73, 106, 157
geog_data_res              = '10m', '5m', '30s', '30s'
dx                         = 27000
dy                         = 27000
map_proj                   = 'lambert'
ref_lat                    = 18.363
ref_lon                    = 78.925
truelat1                   = 18.363
truelat2                   = 18.363
stand_lon                  = 78.925
ref_x                      = 45
ref_y                      = 42.5
geog_data_path             = '/home/hoopoe/ems/wrfems/data/geog'
/
&metgrid
fg_name = 'FILE'
io_form_metgrid = 2,
/
```
- With this the geogrid was executed by ```./geogrid.exe``` and ended with successful completion and following files ```
,geo_em.d02.nc,geo_em.d03.nc,geo_em.d04.nc``` was created. During geogrid.exe, as per the [page](http://ruc.noaa.gov/wrf/WG11/wrf_tutorial_exercises_v35/exercise_1.html) point 10, EORD was processed but no processing was happen for CLAYFRAC and SANDFRAC as said in the page. 

### WPS: Ungrib

- The gfs files downloaded was used, average size around 45MB for each files. The files for time period 00,03, 06 was kept under a folder gfs.
- The vtable was linked inside WPS folder by ```ln -sf ungrib/Variable_Tables/Vtable.GFS Vtable```.
- The GFS files inside the gfs folder is linked with WPS folder by ```./link_grib.csh /home/swl-sacon-dst/wrf/WRF341/WRF_CHEM_trails/coimbatore_nesting/gfs/```, by this following file with links were created by this step inside WPS folder.

```fortran
GRIBFILE.AAA -> /home/swl-sacon-dst/wrf/WRF341/WRF_CHEM_trails/coimbatore_nesting/gfs/14060500.gfs.t00z.pgrb2f00 
GRIBFILE.AAB -> /home/swl-sacon-dst/wrf/WRF341/WRF_CHEM_trails/coimbatore_nesting/gfs/14060500.gfs.t00z.pgrb2f03
GRIBFILE.AAC -> /home/swl-sacon-dst/wrf/WRF341/WRF_CHEM_trails/coimbatore_nesting/gfs/14060500.gfs.t00z.pgrb2f06
```
- The ungrib.exe was run using this command ```./ungrib.exe >& ungrib_data.log```, the log file says successful completion of the ungrib and run for about 1 minutes. It created a 3 files such as this FILE:2010-07-14_00. 

### WPS:Metgrid

- For the metgrid part of WPS, as per the tutorial page the content was checked for namelist.wps and kept as such of geogrid.exe namelist.
- Then the metgrid was executed by ```./metgrid.exe``` and it ended with successful completion and creation of nc files for each domain named from ```met_em.d01.2014-06-05_00:00:00.nc``` to ```met_em.d04.2014-06-05_00:00:00.nc```
- The metgrid.log was reviewed and it is found message saying the EROD field has been read in and written to file.

### WRF:real.exe

- For make a two way nested domain, more idea about nesting is required, so made a visit to WRF EMS user guide chapter 5 and it explains namelist as follows
```bash 
For the uninitiated, 2-way nesting allows for information from a higher resolution child domain to be fed back to its parent domain. The parent domain provides the lateral boundary conditions to the nested domain at each time step and the nested domain is allowed to integrate forward in time with the results fed back onto the parent domain. In the case of 1-way nesting, this feedback mechanism is turned off, so while the parent domain provides the boundary conditions to the nested domain, there is no exchange of information from the nest back to the parent.  
When running the EMS, all nesting is concurrent with the parent domain, meaning the child domain is executed while the parent domain is running. The benefit of concurrent nesting is that the boundary conditions for the child domain are provided at every parent domain time step. The limitation of this method is that it requires more physical memory on your workstation in order to run multiple domains simultaneously.
" 
``` 
- From the above note, it is found that the one way nesting using ndown is computationally less intensive while compared to two way nesting.
- The wrf arw online tutorial gives step by step instruction on running two way and one way nesting and its variant. In which two way nesting with two input files and using meteorological and static data were followed for current purpose.
- Based on this [page](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/CASES/NestRuns/metgrid_2way2inputs.htm) under that tutorial, it is required to change namelist.wps and run metgrid.exe to omit other HR domains (High resolution domains) to form multi temporal met files. It seems one time met files are enough for HR domains and subsequent time metfiles are useless. Considering with this, the metfiles formed are kept as such and removed extra time HR domain metfiles for real.exe.
- For making WRF run, the folder content from ```WRFV3CHEM_serial1/test/em_real ``` was used as such and all the met files are linked inside the em_real folder using the command ```ln -sf /home/swl-sacon-dst/wrf/WRF341/WRF_CHEM_trails/CBE_dustonly/WPS/met_em* .```. With this real.exe was executed and it created wrfinput and wrfbdy files, input file used was of provided with tutorial. As former point leaving HR domains first hour met files remaining links were deleted.
- The namelist.input was prepared by consulting with former [Coimbatore nesting](Working_with_WRF_CHEM_part6.3_WRF_CoimbatoreNesting.md), WRF ARW two way test case [page](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/CASES/NestRuns/wrf_2way2inputs.htm) and wrf chem nesting [tutorial](http://ruc.noaa.gov/wrf/WG11/wrf_tutorial_2012_brazil/wrf-chem_tutorial_nesting.pdf) and WRF EMS namelist for Coimbatore four nest domain file namelist.real.
- The edited namelist was [this](namelist.input_2way_CBEDUSTONLY) and executed ./real.exe. Wrong entry in end_hour and interval_seconds under &time_control results in running of real.exe with successful completion of only producing wrfinput_01 but no wrfbdy_01. Based on [this](http://forum.wrfforum.com/viewtopic.php?f=10&t=419) it was came to know and by editing the errors lets to create wrfbdy_01 in ./real.exe along with wrfinput_01, wrfinput_02, wrfinput_03 and wrfinput_04. There where no wrfbdy for nested domains as it is a separate process by ndown.exe or downscaling in two way nesting.  

#### WRF:wrf.exe

19. The wrf.exe was executed after real.exe. It complained about non availability of wrfchemi file and it was rectified by changing the namlist &time_control entry ``` io_form_auxinput5 =0``` instead of 2. It also give error of cu_physics has to change into 5 and cu_diag into 1. Although wrf.exe ends with memory dumped error. Indicating two way nesting computational requirement is far above the running HP ProBook 4540s laptop. It was followed by running this case with parallel compiled WRF chem and it was also a memory related error. 
